In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.utils import class_weight
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix 
print("TensorFlow version:", tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from datetime import datetime
from sklearn.metrics import accuracy_score
import gc
import warnings
warnings.filterwarnings('ignore')

TensorFlow version: 2.12.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14291168666452224932
xla_global_id: -1
]


In [4]:
RANDOM_SEED = 42

In [5]:
tf.test.is_built_with_cuda()

False

In [6]:
def nan_click_viewed__deleting(X, y = None):
    X = X[X['click'].isna() == False]
    X = X[X['view'] != 0]
    return X 

In [7]:
def dub_dropping(X, y = None):
    X =  X.drop_duplicates()                              
    return X

In [8]:
def nan_filling(X, y = None):
    nan_cols = {}
    for col in X.columns:
        nan_count = X[col].isna().sum()
        if nan_count != 0:
            per = np.round((nan_count / df.shape[0]) * 100, 2)
            nan_cols[col] = per
                       
    for col, per in nan_cols.items():
        if per < 5:
            X = X[X[col].isna() == False]
        else:
            X[col] = X[col].fillna('unknown')
            
    return X

In [9]:
df_clickhouse = pd.read_csv('../internship/clickhouse.csv')
df_clickhouse.shape

(5889167, 39)

In [10]:
df_creatives = pd.read_csv('../internship/creatives.csv')
df_creatives.shape

(132, 15)

In [11]:
df_creatives = df_creatives.rename(columns={'id': 'creative_id'})

In [12]:
df = pd.merge(df_clickhouse, df_creatives, on="creative_id")
df.shape

(5889167, 53)

In [13]:
del df_clickhouse
del df_creatives
gc.collect()

63

In [14]:
df = nan_filling(dub_dropping(nan_click_viewed__deleting(df)))

In [15]:
df_train, df_test = train_test_split(df, stratify = df['click'], shuffle=True, test_size=0.2, random_state=RANDOM_SEED)

X_train = df_train.drop('click', axis = 1)
y_train = df_train['click']

X_test = df_test.drop('click', axis = 1)
y_test = df_test['click']

In [200]:
# X = df.drop('click', axis = 1)
# y = df['click']

In [16]:
del df
del df_train
del df_test
gc.collect()

0

In [17]:
def column_deleting(X, y = None):
    columns_for_drop = ['auction_date_time', 'impression_hash', 'ssp', 'is_pay',  'pay_price', 'auction_id', 'impression_id', 'bid_id', 'auction_type', 'bid_floor', 'bid_price', 'is_win', 'view', 'place_number', 'ssp_user_id', 'campaign_id_x', 'enter_utm_campaign',  'stream_id_x', 'link_id', 'format', 'ip_v4', 'ip_v6', 'tag_id', 'iab_category_x', 'ab_test', 'campaign_id_y', 'user_id', 'stream_id_y', 'is_deleted', 'image', 'created_at', 'updated_at']
    X = X.drop(columns_for_drop, axis = 1)
    X.reset_index(drop=True, inplace=True)
    return X

In [18]:
def feature_reduction(X, y = None):
    X['browser'] = X['browser'].apply(lambda x : 'Chrome' if x == 'Google' else x)
    X['browser'] = X['browser'].apply(lambda x : 'Yandex Browser' if x == 'Yandex' else x)
    X.reset_index(drop=True, inplace=True)
    return X

In [19]:
def date_time_col_transformation(X, y = None):
    X['req_date_time'] = pd.to_datetime(X['event_date_time'], format='%Y-%m-%d %H:%M:%S')
    X['day'] =  X['req_date_time'].dt.day
    X['month'] =  X['req_date_time'].dt.month
    X['year'] =  X['req_date_time'].dt.year
    X['hour'] =  X['req_date_time'].dt.hour
    X['day_of_week'] = X['event_date_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday())
    X['is_holyday'] = X['day_of_week'].apply(lambda x : 1 if x in [5 , 6] else 0)
    X['is_friday'] = X['day_of_week'].apply(lambda x : 1 if x == 4 else 0)
    X['is_monday'] = X['day_of_week'].apply(lambda x : 1 if x == 0 else 0)
    X['day_part'] = X['hour'].apply(lambda x : 'Night' if x >= 22 and x < 5 else ('Morning' if x >= 5 and x < 12 else ('Day' if x >= 12 and x < 17 else 'Evening')))
    X['year_part'] = X['month'].apply(lambda x : 'Winter' if x in [12, 1, 2] else ('Spring' if x in [3, 4, 5] else ('Summer' if x in [6, 7, 8] else 'Autumn')))
    X = X.drop(['event_date_time', 'req_date_time', 'year'], axis = 1)
    X.reset_index(drop=True, inplace=True)
    return X

In [25]:
def type_transformation(X, y = None):
    X['loss_reason'] = X['loss_reason'].astype('int8')
    X['device'] = X['device'].astype('int8')
    X['day'] = X['day'].astype('int8')
    X['month'] = X['month'].astype('int8')
    X['hour'] = X['hour'].astype('int8')
    X['day_of_week'] = X['day_of_week'].astype('int8')
    X['is_holyday'] = X['is_holyday'].astype('int8')
    X['is_friday'] = X['is_friday'].astype('int8')
    X['is_monday'] = X['is_monday'].astype('int8')
    X.reset_index(drop=True, inplace=True)
    return X

In [32]:
ohe = OneHotEncoder(sparse = False, max_categories = 50, handle_unknown = 'ignore')
#ohe = OneHotEncoder(sparse = True, handle_unknown = 'ignore')

In [33]:
def cat_encoding(X, y = None):
    cat_cols = X.columns
    ohe.fit(X[cat_cols])
    ft = ohe.transform(X[cat_cols])
    df_ft = pd.DataFrame(ft, columns=ohe.get_feature_names_out())
    X.reset_index(drop=True, inplace=True)
    df_ft.reset_index(drop=True, inplace=True)
    df_ft = df_ft.astype('int8')
    X = pd.concat([X, df_ft], axis=1)
    X.drop(cat_cols, inplace = True, axis = 1)
    
    return X

In [21]:
def cat_encoding(X, y = None):
    cat_cols = X.columns
    ohe.fit(X[cat_cols])
    ft = ohe.transform(X[cat_cols])
    # df_ft = pd.DataFrame(ft, columns=ohe.get_feature_names_out())
    # X.reset_index(drop=True, inplace=True)
    # df_ft.reset_index(drop=True, inplace=True)
    # df_ft = df_ft.astype('int8')
    # X = pd.concat([X, df_ft], axis=1)
    # X.drop(cat_cols, inplace = True, axis = 1)
    X = ft
    return X

In [34]:
preprocessor = Pipeline(
    steps=
    [
        ('column_deleting', FunctionTransformer(func=column_deleting)),
        ('feature_reduction', FunctionTransformer(func=feature_reduction)),
        ('date_time_col_transformation', FunctionTransformer(func=date_time_col_transformation)),
        ('type_transformation', FunctionTransformer(func=type_transformation)),
        ('cat_encoding', FunctionTransformer(func=cat_encoding)),
        
    ]
)

In [38]:
import pickle

In [210]:
# X = preprocessor.fit_transform(X, y)
# train_cols = X.columns
# train_cols_dict['train_cols'] = train_cols

In [211]:
# train_cols_dict['train_cols']

In [212]:
# with open('train_cols_dict.pkl', 'wb') as file:
#     pickle.dump(train_cols_dict, file)

In [41]:
with open('train_cols_dict.pkl', 'rb') as f:
        train_cols_dict = pickle.load(f)

train_cols = train_cols_dict['train_cols']
train_cols

Index(['loss_reason_0', 'loss_reason_1', 'loss_reason_2', 'loss_reason_4',
       'loss_reason_5', 'loss_reason_6', 'loss_reason_9', 'loss_reason_10',
       'loss_reason_11', 'loss_reason_12',
       ...
       'is_holyday_0', 'is_holyday_1', 'is_friday_0', 'is_friday_1',
       'is_monday_0', 'is_monday_1', 'day_part_Day', 'day_part_Evening',
       'day_part_Morning', 'year_part_Winter'],
      dtype='object', length=380)

In [46]:
X_train_prep = preprocessor.fit_transform(X_train, y_train)
X_train_prep.shape

(2392377, 380)

In [43]:
# X_train_prep_ = pd.DataFrame(columns=train_cols)
# X_train_prep_[X_train_prep.columns] = X_train_prep
# X_train_prep_ = X_train_prep_.fillna(0)
# X_train_prep_.shape

(2392377, 381)

In [47]:
y_train.shape

(2392377,)

In [45]:
# X_train_prep_.drop('enter_utm_term_30347_27771', axis = 1, inplace=True)
# X_train_prep_.shape

(2392377, 380)

In [48]:
X_test_prep = preprocessor.fit_transform(X_test, y_test)
X_test_prep.shape

(598095, 364)

In [49]:
y_test.shape

(598095,)

In [50]:
X_test_prep_ = pd.DataFrame(columns=train_cols)
X_test_prep_[X_test_prep.columns] = X_test_prep
X_test_prep_ = X_test_prep_.fillna(0)
X_test_prep_.shape

(598095, 380)

In [81]:
X_train_tensor = tf.convert_to_tensor(X_test_prep)
y_train_tensor = tf.convert_to_tensor(y_train_)

X_test_tensor = tf.convert_to_tensor(X_test_prep_)
y_test_tensor = tf.convert_to_tensor(y_test)

In [78]:
import keras

In [79]:
y_train_ = keras.utils.to_categorical(y_train, len(y_train.unique()))
y_test_ = keras.utils.to_categorical(y_test, len(y_test.unique()))

In [67]:
X_train_tensor.shape

TensorShape([598095, 364])

In [82]:
y_train_tensor.shape

TensorShape([2392377, 2])

In [68]:
X_test_tensor.shape

TensorShape([598095, 380])

In [64]:
kf = KFold(n_splits = 2, shuffle = True, random_state = RANDOM_SEED)

In [70]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(100, input_dim = X_train_tensor.shape[1], activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(len(y_train.unique()))
])

In [71]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

In [ ]:
class_weights = class_weight.compute_class_weight(
    class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train  
   )
class_weights = dict(enumerate(class_weights))

In [72]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy']
             # metrics=[tf.keras.metrics.BinaryCrossentropy()]
              #metrics = [tf.keras.metrics.AUC()]
             )

In [75]:
model.fit(X_train_tensor, y_train, batch_size=1024, shuffle=True, epochs = 3) #class_weight=class_weights,

ValueError: Data cardinality is ambiguous:
  x sizes: 598095
  y sizes: 2392377
Make sure all arrays contain the same number of samples.

In [57]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [59]:
predictions = model(X_test_prep_)
predictions

AttributeError: Exception encountered when calling layer 'sequential' (type Sequential).

'tuple' object has no attribute 'rank'

Call arguments received by layer 'sequential' (type Sequential):
  • inputs=        loss_reason_0  loss_reason_1  loss_reason_2  loss_reason_4   
0                   0              0              0              0  \
1                   0              0              0              0   
2                   0              0              1              0   
3                   0              0              0              0   
4                   0              0              0              0   
...               ...            ...            ...            ...   
598090              0              0              0              0   
598091              0              1              0              0   
598092              0              0              1              0   
598093              0              0              0              0   
598094              0   

In [60]:
probs = probability_model(X_test_prep_)
probs

AttributeError: Exception encountered when calling layer 'sequential' (type Sequential).

'tuple' object has no attribute 'rank'

Call arguments received by layer 'sequential' (type Sequential):
  • inputs=        loss_reason_0  loss_reason_1  loss_reason_2  loss_reason_4   
0                   0              0              0              0  \
1                   0              0              0              0   
2                   0              0              1              0   
3                   0              0              0              0   
4                   0              0              0              0   
...               ...            ...            ...            ...   
598090              0              0              0              0   
598091              0              1              0              0   
598092              0              0              1              0   
598093              0              0              0              0   
598094              0   

In [ ]:
roc_auc_score(y_test, probs)

In [266]:
def baseline_model(hidden_layer_dim):

    n_features_in_ = 100
    X_shape_ = X_train_tensor.shape
    n_classes_ = len(y_train.unique())
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(n_features_in_, input_dim = X_shape_[1], activation='relu'),
    tf.keras.layers.Dense(hidden_layer_dim, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Dense(n_classes_, activation='softmax'),
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=loss_fn,
              #metrics=[tf.keras.metrics.Accuracy()],
              metrics=['accuracy'],
              #weighted_metrics=class_weights,
             )
    return model


In [267]:
estimator = KerasClassifier(baseline_model, loss = loss_fn, hidden_layer_dim=128, verbose=0)

In [268]:
estimator.get_params().keys()

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_dim', 'class_weight'])

In [272]:
estimator.fit(X_train_tensor, y_train,
             batch_size=1024,
             epochs=3,
             #class_weight=class_weights,
             shuffle=True
             )

KerasClassifier(
	model=<function baseline_model at 0x0000025C9AE0A200>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=<keras.losses.BinaryCrossentropy object at 0x0000025BA607F6D0>
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_dim=128
	class_weight=None
)

In [276]:
model = baseline_model(hidden_layer_dim=128)
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [278]:
estimator = KerasClassifier(probability_model, loss = loss_fn, verbose=0)

In [281]:
estimator.fit(X_train_tensor, y_train,
             batch_size=1024,
             epochs=3,
             #class_weight=class_weights,
             shuffle=True
             )

KerasClassifier(
	model=<keras.engine.sequential.Sequential object at 0x0000025AFAB92450>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=<keras.losses.BinaryCrossentropy object at 0x0000025BA607F6D0>
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [280]:
y_proba = estimator.predict(X_test_tensor)
y_proba

NotFittedError: Estimator needs to be fit before `predict` can be called